# 1 - Simple Sentiment Analysis

Reference : https://github.com/bentrevett/pytorch-sentiment-analysis

In this series we'll be building a machine learning model to detect sentiment (i.e. detect if a sentence is positive or negative) using PyTorch. This will be done on Korean movie reviews, using the [Naver sentiment movie corpus v1.0](https://github.com/e9t/nsmc).

In this first notebook, we'll start very simple to understand the general concepts whilst not really caring about good results. Further notebooks will build on this knowledge and we'll actually get good results.

### Introduction

We'll be using a **recurrent neural network** (RNN) as they are commonly used in analysing sequences. An RNN takes in sequence of words, $X=\{x_1, ..., x_T\}$, one at a time, and produces a _hidden state_, $h$, for each word. We use the RNN _recurrently_ by feeding in the current word $x_t$ as well as the hidden state from the previous word, $h_{t-1}$, to produce the next hidden state, $h_t$. 

$$h_t = \text{RNN}(x_t, h_{t-1})$$

Once we have our final hidden state, $h_T$, (from feeding in the last word in the sequence, $x_T$) we feed it through a linear layer, $f$, (also known as a fully connected layer), to receive our predicted sentiment, $\hat{y} = f(h_T)$.

Below shows an example sentence, with the RNN predicting zero, which indicates a negative sentiment. The RNN is shown in orange and the linear layer shown in silver. Note that we use the same RNN for every word, i.e. it has the same parameters. The initial hidden state, $h_0$, is a tensor initialized to all zeros. 

![](assets/sentiment1.png)

**Note:** some layers and steps have been omitted from the diagram, but these will be explained later.

## Preparing Data

The Naver sentiment movie corpus consists of 200,000 movie reviews, each marked as being a positive or negative review.

#### Data description

- Each file is consisted of three columns: `id`, `document`, `label`
    - `id`: The review id, provieded by Naver
    - `document`: The actual review
    - `label`: The sentiment class of the review. (0: negative, 1: positive)
    - Columns are delimited with tabs (i.e., `.tsv` format; but the file extension is `.txt` for easy access for novices)
- 200K reviews in total
    - `ratings.txt`: All 200K reviews
    - `ratings_test.txt`: 50K reviews held out for testing
    - `ratings_train.txt`: 150K reviews for training

#### Characteristics

- All reviews are shorter than 140 characters
- Each sentiment class is sampled equally (i.e., random guess yields 50% accuracy)
    - 100K negative reviews (originally reviews of ratings 1-4)
    - 100K positive reviews (originally reviews of ratings 9-10)
    - Neutral reviews (originally reviews of ratings 5-8) are excluded
    


The Naver sentiment movie corpus only has train/test splits, so we need to create a validation set. We can do this with the `split_ratio` argument. Mecab tokenizer is used when building dataset.

In [1]:
import torch
from data_utils import build_dataset

In [2]:
torch.cuda.is_available()

True

In [3]:
train_dataset, valid_dataset = build_dataset('data/ratings_train.txt', split_ratio=0.7)

[Info] Get 150000 data from data/ratings_train.txt
[Info] Drop null data, now the length of this data is 149995
[Info] Tokenize...
[Info] Build vocabulary
[Info] Vocabulary size= 55332
[Info] Split 149995 data to 104996 for train data,  44999 for valid data.
[Info] Build datasets


We can see how many examples are in each split by checking their length.

In [4]:
print(f'Number of training examples: {len(train_dataset)}')
print(f'Number of testing examples: {len(valid_dataset)}')

Number of training examples: 104996
Number of testing examples: 44999


We can also check an example.

In [5]:
print(train_dataset[0])

(tensor([25052, 47246,  6247, 13914, 47525, 20471, 25724]), tensor([1]))


Train dataset and valid dataset share vocabulary. We use this VOCAB to make test_dataset with the `predefined_vocab` argument. For reusing, we will save all the dataset we made. 

In [6]:
assert train_dataset.vocab == valid_dataset.vocab

VOCAB = train_dataset.vocab
print(f"Unique tokens in TEXT vocabulary: {len(VOCAB)}")

Unique tokens in TEXT vocabulary: 55332


In [7]:
print([word for word, idx in VOCAB.items() if idx in train_dataset[0][0]])

['남', '는다', '에게', '여운', '는', '이', '나']


In [8]:
test_dataset = build_dataset('data/ratings_test.txt', predefined_vocab=VOCAB)

torch.save((train_dataset, valid_dataset, test_dataset), 'dataset.pt')

[Info] Get 50000 data from data/ratings_test.txt
[Info] Drop null data, now the length of this data is 49997
[Info] Tokenize...
[Info] Pre-defined vocabulary found.
[Info] Vocabulary size= 55332
[Info] Build dataset


In [9]:
from data_utils import build_dataloader

BATCH_SIZE = 3

train_iterator = build_dataloader(train_dataset, batch_size = BATCH_SIZE)
valid_iterator = build_dataloader(valid_dataset, batch_size = BATCH_SIZE)
test_iterator = build_dataloader(test_dataset, batch_size = BATCH_SIZE)

## Build the Model

The next stage is building the model that we'll eventually train and evaluate. 

There is a small amount of boilerplate code when creating models in PyTorch, note how our `RNN` class is a sub-class of `nn.Module` and the use of `super`.

Within the `__init__` we define the _layers_ of the module. Our three layers are an _embedding_ layer, our RNN, and a _linear_ layer. All layers have their parameters initialized to random values, unless explicitly specified.

The embedding layer is used to transform our sparse one-hot vector (sparse as most of the elements are 0) into a dense embedding vector (dense as the dimensionality is a lot smaller and all the elements are real numbers). This embedding layer is simply a single fully connected layer. As well as reducing the dimensionality of the input to the RNN, there is the theory that words which have similar impact on the sentiment of the review are mapped close together in this dense vector space. For more information about word embeddings, see [here](https://monkeylearn.com/blog/word-embeddings-transform-text-numbers/).

The RNN layer is our RNN which takes in our dense vector and the previous hidden state $h_{t-1}$, which it uses to calculate the next hidden state, $h_t$.

![](assets/sentiment7.png)

Finally, the linear layer takes the final hidden state and feeds it through a fully connected layer, $f(h_T)$, transforming it to the correct output dimension.

The `forward` method is called when we feed examples into our model.

Each batch, `text`, is a tensor of size _**[batch size, sentence length]**_. That is a batch of sentences, each having each word converted into a one-hot vector. 

You may notice that this tensor should have another dimension due to the one-hot vectors, however PyTorch conveniently stores a one-hot vector as it's index value, i.e. the tensor representing a sentence is just a tensor of the indexes for each token in that sentence. The act of converting a list of tokens into a list of indexes is commonly called *numericalizing*.

The input batch is then passed through the embedding layer to get `embedded`, which gives us a dense vector representation of our sentences. `embedded` is a tensor of size _**[batch size, sentence length, embedding dim]**_.

`embedded` is then fed into the RNN. In some frameworks you must feed the initial hidden state, $h_0$, into the RNN, however in PyTorch, if no initial hidden state is passed as an argument it defaults to a tensor of all zeros.

The RNN returns 2 tensors, `output` of size _**[batch size, sentence length, hidden dim]**_ and `hidden` of size _**[batch size, 1, hidden dim]**_. `output` is the concatenation of the hidden state from every time step, whereas `hidden` is simply the final hidden state. We verify this using the `assert` statement. Note the `squeeze` method, which is used to remove a dimension of size 1. 

Finally, we feed the last hidden state, `hidden`, through the linear layer, `fc`, to produce a prediction.

In [10]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim): 
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first = True)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
        
        #embedded = [batch size, sent len, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [batch size, sent len, hid dim]
        #hidden = [batch size, 1, hid dim]
        
        return self.fc(hidden.squeeze(0))

We now create an instance of our RNN class. 

The input dimension is the dimension of the one-hot vectors, which is equal to the vocabulary size. 

The embedding dimension is the size of the dense word vectors. This is usually around 50-250 dimensions, but depends on the size of the vocabulary.

The hidden dimension is the size of the hidden states. This is usually around 100-500 dimensions, but also depends on factors such as on the vocabulary size, the size of the dense vectors and the complexity of the task.

The output dimension is usually the number of classes, however in the case of only 2 classes the output value is between 0 and 1 and thus can be 1-dimensional, i.e. a single scalar real number.

In [11]:
INPUT_DIM = len(VOCAB)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

Let's also create a function that will tell us how many trainable parameters our model has so we can compare the number of parameters across different models.

In [12]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 5,625,105 trainable parameters


## Train the Model

Now we'll set up the training and then train the model.

First, we'll create an optimizer. This is the algorithm we use to update the parameters of the module. Here, we'll use _stochastic gradient descent_ (SGD). The first argument is the parameters will be updated by the optimizer, the second is the learning rate, i.e. how much we'll change the parameters by when we do a parameter update.

In [13]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=1e-3)

Next, we'll define our loss function. In PyTorch this is commonly called a criterion. 

The loss function here is _binary cross entropy with logits_. 

Our model currently outputs an unbound real number. As our labels are either 0 or 1, we want to restrict the predictions to a number between 0 and 1. We do this using the _sigmoid_ or _logit_ functions. 

We then use this this bound scalar to calculate the loss using binary cross entropy. 

The `BCEWithLogitsLoss` criterion carries out both the sigmoid and the binary cross entropy steps.

In [14]:
criterion = nn.BCEWithLogitsLoss()

Using `.to`, we can place the model and the criterion on the GPU (if we have one). 

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [16]:
model = model.to(device)
criterion = criterion.to(device)

Our criterion function calculates the loss, however we have to write our function to calculate the accuracy. 

This function first feeds the predictions through a sigmoid layer, squashing the values between 0 and 1, we then round them to the nearest integer. This rounds any value greater than 0.5 to 1 (a positive sentiment) and the rest to 0 (a negative sentiment).

We then calculate how many rounded predictions equal the actual labels and average it across the batch.

In [17]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

The `train` function iterates over all examples, one batch at a time. 

`model.train()` is used to put the model in "training mode", which turns on _dropout_ and _batch normalization_. Although we aren't using them in this model, it's good practice to include it.

For each batch, we first zero the gradients. Each parameter in a model has a `grad` attribute which stores the gradient calculated by the `criterion`. PyTorch does not automatically remove (or "zero") the gradients calculated from the last gradient calculation, so they must be manually zeroed.

We then feed the batch of sentences, `batch_sentence`, into the model. Note, you do not need to do `model.forward(batch_sentence)`, simply calling the model works. The `squeeze` is needed as the predictions are initially size _**[batch size, 1]**_, and we need to remove the dimension of size 1 as PyTorch expects the predictions input to our criterion function to be of size _**[batch size]**_.

The loss and accuracy are then calculated using our predictions and the labels, `batch_label`, with the loss being averaged over all examples in the batch.

We calculate the gradient of each parameter with `loss.backward()`, and then update the parameters using the gradients and optimizer algorithm with `optimizer.step()`.

The loss and accuracy is accumulated across the epoch, the `.item()` method is used to extract a scalar from a tensor which only contains a single value.

Finally, we return the loss and accuracy, averaged across the epoch. The `len` of an iterator is the number of batches in the iterator.

In [18]:
from tqdm import tqdm

In [19]:
def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch_sentence, batch_label in tqdm(iterator):
        batch_sentence = batch_sentence.to(device)
        batch_label = batch_label.to(device)
        
        optimizer.zero_grad()
                
        predictions = model(batch_sentence).squeeze(1)
       
        batch_label = batch_label.float()
        loss = criterion(predictions, batch_label)
        
        acc = binary_accuracy(predictions, batch_label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

`evaluate` is similar to `train`, with a few modifications as you don't want to update the parameters when evaluating.

`model.eval()` puts the model in "evaluation mode", this turns off _dropout_ and _batch normalization_. Again, we are not using them in this model, but it is good practice to include them.

No gradients are calculated on PyTorch operations inside the `with no_grad()` block. This causes less memory to be used and speeds up computation.

The rest of the function is the same as `train`, with the removal of `optimizer.zero_grad()`, `loss.backward()` and `optimizer.step()`, as we do not update the model's parameters when evaluating.

In [21]:
def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch_sentence, batch_label in tqdm(iterator):
            batch_sentence = batch_sentence.to(device)
            batch_label = batch_label.to(device)
            
            predictions = model(batch_sentence).squeeze(1)
            
            batch_label = batch_label.float()
            loss = criterion(predictions, batch_label)
            
            acc = binary_accuracy(predictions, batch_label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

We'll also create a function to tell us how long an epoch takes to compare training times between models.

In [22]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

We then train the model through multiple epochs, an epoch being a complete pass through all examples in the training and validation sets.

At each epoch, if the validation loss is the best we have seen so far, we'll save the parameters of the model and then after training has finished we'll use that model on the test set.

In [24]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

  0%|          | 17/34999 [00:00<03:26, 169.17it/s]

Epoch: 01 | Epoch Time: 2m 25s
	Train Loss: 0.635 | Train Acc: 63.92%
	 Val. Loss: 0.685 |  Val. Acc: 52.55%


  0%|          | 21/34999 [00:00<02:51, 204.18it/s]

Epoch: 02 | Epoch Time: 2m 24s
	Train Loss: 0.560 | Train Acc: 72.78%
	 Val. Loss: 0.526 |  Val. Acc: 77.50%


  0%|          | 18/34999 [00:00<03:17, 176.80it/s]

Epoch: 03 | Epoch Time: 2m 22s
	Train Loss: 0.508 | Train Acc: 78.09%
	 Val. Loss: 0.503 |  Val. Acc: 80.05%


  0%|          | 16/34999 [00:00<03:44, 155.53it/s]

Epoch: 04 | Epoch Time: 2m 24s
	Train Loss: 0.519 | Train Acc: 76.52%
	 Val. Loss: 0.489 |  Val. Acc: 79.54%


100%|██████████| 15000/15000 [00:15<00:00, 967.94it/s]

Epoch: 05 | Epoch Time: 2m 18s
	Train Loss: 0.501 | Train Acc: 78.37%
	 Val. Loss: 0.598 |  Val. Acc: 64.57%


You may have noticed the loss is not really decreasing and the accuracy is poor. This is due to several issues with the model which we'll improve in the next notebook.

Finally, the metric we actually care about, the test loss and accuracy, which we get from our parameters that gave us the best validation loss.

In [25]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

100%|██████████| 16666/16666 [00:17<00:00, 962.31it/s]

Test Loss: 0.497 | Test Acc: 79.12%


## Next Steps

In the next notebook, the improvements we will make are:
- packed padded sequences
- pre-trained word embeddings
- different RNN architecture
- bidirectional RNN
- multi-layer RNN
- regularization
- a different optimizer

This will allow us to achieve ~84% accuracy.